#Load a large set of expense transactions

In [8]:
# I have the the cleaned data from python(week2) in drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TransactionVolume").getOrCreate()

In [17]:
df = spark.read.csv('/content/drive/My Drive/cleaned_expenses.csv', header=True, inferSchema=True)
from pyspark.sql.functions import date_format
df = df.withColumn("month", date_format("expense_date", "yyyy-MM"))
df.show()

+-------+-----------+-------+------------+--------------------+-------+
|user_id|category_id| amount|expense_date|         description|  month|
+-------+-----------+-------+------------+--------------------+-------+
|      1|          1| 2500.0|  2025-07-01|Weekly groceries ...|2025-07|
|      2|          2|10000.0|  2025-07-01|  Monthly house rent|2025-07|
|      3|          5| 1200.0|  2025-07-02| Dinner with friends|2025-07|
|      4|          3| 1800.0|  2025-03-07|Electricity and w...|2025-03|
|      5|          6| 2200.0|  2025-07-04|General health ch...|2025-07|
|      6|          4|  800.0|  2025-07-04|Local transport c...|2025-07|
|      7|          7| 5000.0|  2025-07-05|        Tuition fees|2025-07|
|      8|          8|  950.0|  2025-07-05|    Movie and snacks|2025-07|
|      1|          5|  700.0|  2025-07-06|       Lunch at cafe|2025-07|
|      2|          1| 3000.0|  2025-07-07|Vegetables and fr...|2025-07|
|      3|          4|  600.0|  2025-07-08|   Bus pass recharge|2

#Group by user to calculate total monthly spend

In [19]:
from pyspark.sql.functions import col, sum
df = df.withColumn("amount", col("amount").cast("float"))
monthly_spend = df.groupBy("user_id", "month").agg(sum("amount").alias("total_spent")).orderBy("user_id","month")
monthly_spend.show()

+-------+-------+-----------+
|user_id|  month|total_spent|
+-------+-------+-----------+
|      1|2025-07|     4150.0|
|      2|2025-07|    14400.0|
|      3|2025-07|     4100.0|
|      4|2025-03|     1800.0|
|      4|2025-07|     4400.0|
|      5|2025-07|    13600.0|
|      6|2025-07|     4800.0|
|      6|2025-11|     1500.0|
|      7|2025-07|     8600.0|
|      8|2025-07|    13750.0|
+-------+-------+-----------+



#Detect unusual spikes or large one-time expenses

In [29]:
# Calculate average monthly spend per user
from pyspark.sql.functions import when, avg
avg_spend = monthly_spend.groupBy("user_id").agg(avg("total_spent").alias("avg_monthly_spend"))
joined = monthly_spend.join(avg_spend, on="user_id")
spikes = joined.withColumn("spike", when(col("total_spent") > 1.5 * col("avg_monthly_spend"), "Yes").otherwise("No"))
spikes.show()

+-------+-------+-----------+-----------------+-----+
|user_id|  month|total_spent|avg_monthly_spend|spike|
+-------+-------+-----------+-----------------+-----+
|      1|2025-07|     4150.0|           4150.0|   No|
|      6|2025-07|     4800.0|           3150.0|  Yes|
|      6|2025-11|     1500.0|           3150.0|   No|
|      3|2025-07|     4100.0|           4100.0|   No|
|      5|2025-07|    13600.0|          13600.0|   No|
|      4|2025-07|     4400.0|           3100.0|   No|
|      4|2025-03|     1800.0|           3100.0|   No|
|      8|2025-07|    13750.0|          13750.0|   No|
|      7|2025-07|     8600.0|           8600.0|   No|
|      2|2025-07|    14400.0|          14400.0|   No|
+-------+-------+-----------+-----------------+-----+



In [30]:
spikes.filter(col("spike") == "Yes").show()

+-------+-------+-----------+-----------------+-----+
|user_id|  month|total_spent|avg_monthly_spend|spike|
+-------+-------+-----------+-----------------+-----+
|      6|2025-07|     4800.0|           3150.0|  Yes|
+-------+-------+-----------+-----------------+-----+

